# Data Science Pandas - Joins, Time Series & Rolling Statistics

## Tasks Today:

1) <b>Pandas</b> <br>
 &nbsp;&nbsp;&nbsp;&nbsp; a) Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - merge() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Breaking Down the Inner Join <br>
 &nbsp;&nbsp;&nbsp;&nbsp; b) Outer Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Use Case Example <br>
 &nbsp;&nbsp;&nbsp;&nbsp; c) Rolling Statistics and Time Series w/Pandas <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - read_csv() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - shift() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Sending and Reading from SQL database <br>

In [1]:
import pandas as pd
import numpy as np

### Joins <br>
<p>Very often you will have to combine data from several different sources to obtain the actual dataset you need for your exploration or modeling. Pandas draws heavily on SQL in its API design for joins, and if you are familiar with SQL joins, then Pandas will come naturally. Imagine you have two tables, one with customer id, city, and age and the other with their feedback:</p>

In [3]:
# Generate some fake data
city_data = {
    1: 'Chicago',
    2: 'Pittsburgh',
    3: 'Buffalo',
    4: 'Orlando',
    5: 'Boston',
    6: 'Phoenix'
}


ages = np.random.randint(18, 75, 100)
city = [city_data[num] for num in np.random.randint(1, 7, 100)]
customer_id = np.arange(1000, 1100)



<p>Let's assume we have a dataset containing the customer ID's cities, and ages for a set of customers. Suppose further, that we have a second table, containing some customer ID's, and a feedback survey rating from those customers. Suppose finally, that you as an analyst are asked the following question: What city tends to give the best feedback for our product? We'll need a join operation to get the answer.</p>

In [7]:
customer_demographics = pd.DataFrame.from_dict({
    'age': ages,
    'city': city,
    'customer_id': customer_id
})

customer_demographics

,age,city,customer_id
0,26,Orlando,1000
1,25,Chicago,1001
2,50,Buffalo,1002
3,22,Boston,1003
4,23,Phoenix,1004
...,...,...,...
95,49,Buffalo,1095
96,18,Pittsburgh,1096
97,22,Boston,1097
98,29,Phoenix,1098


In [10]:
customer_demographics.head(15)

,age,city,customer_id
0,26,Orlando,1000
1,25,Chicago,1001
2,50,Buffalo,1002
3,22,Boston,1003
4,23,Phoenix,1004
5,61,Phoenix,1005
6,36,Buffalo,1006
7,61,Boston,1007
8,33,Chicago,1008
9,58,Orlando,1009


In [8]:
# Generate some other fake data
customer_id = np.random.randint(1000, 1200, 100)
customer_rating = np.random.randint(1,11, 100)

surveys = pd.DataFrame.from_dict({
    'customer_id': customer_id,
    'rating': customer_rating
})

In [9]:
surveys

,customer_id,rating
0,1010,5
1,1033,4
2,1164,5
3,1071,2
4,1042,7
...,...,...
95,1009,2
96,1021,5
97,1028,5
98,1093,1


##### merge()

<p>We can zip the two dataframes up with an inner join, using the Pandas API as follows:</p>

In [13]:
# SELECT *
# FROM customer_demographics
# (INNER) JOIN surveys
# ON customer_demographics.customer_id = surveys.customer_id

demo_ratings = customer_demographics.merge(surveys, how='inner', on='customer_id')

demo_ratings

,age,city,customer_id,rating
0,22,Boston,1003,3
1,22,Boston,1003,9
2,23,Phoenix,1004,3
3,23,Phoenix,1004,9
4,61,Phoenix,1005,5
5,36,Buffalo,1006,6
6,58,Orlando,1009,2
7,38,Pittsburgh,1010,5
8,70,Chicago,1012,4
9,52,Buffalo,1013,4


**.join() method**

In [16]:
surveys.set_index('customer_id')

,rating
customer_id,
1010,5
1033,4
1164,5
1071,2
1042,7
...,...
1009,2
1021,5
1028,5


In [15]:
demo_surveys_join = customer_demographics.join(other=surveys.set_index('customer_id'), on='customer_id', how='inner')
demo_surveys_join

,age,city,customer_id,rating
3,22,Boston,1003,3
3,22,Boston,1003,9
4,23,Phoenix,1004,3
4,23,Phoenix,1004,9
5,61,Phoenix,1005,5
6,36,Buffalo,1006,6
9,58,Orlando,1009,2
10,38,Pittsburgh,1010,5
12,70,Chicago,1012,4
13,52,Buffalo,1013,4


In [19]:
demographics_customer_index = customer_demographics.set_index('customer_id')
survey_cust = surveys.set_index('customer_id')

demographics_customer_index.join(survey_cust, how='inner')

,age,city,rating
customer_id,,,
1003,22,Boston,3
1003,22,Boston,9
1004,23,Phoenix,3
1004,23,Phoenix,9
1005,61,Phoenix,5
1006,36,Buffalo,6
1009,58,Orlando,2
1010,38,Pittsburgh,5
1012,70,Chicago,4


Remove any duplicates

In [21]:
demo_ratings_drop_dups = demo_ratings.drop_duplicates('customer_id', keep='last').reset_index(drop=True)
demo_ratings_drop_dups

,age,city,customer_id,rating
0,22,Boston,1003,9
1,23,Phoenix,1004,9
2,61,Phoenix,1005,5
3,36,Buffalo,1006,6
4,58,Orlando,1009,2
5,38,Pittsburgh,1010,5
6,70,Chicago,1012,4
7,52,Buffalo,1013,4
8,22,Orlando,1014,10
9,39,Buffalo,1017,7


<p>Now it's a simple groupby operation to reveal the answer:</p>

In [33]:
city_ages_ratings = demo_ratings_drop_dups.groupby('city').mean().round(decimals=2)[['age', 'rating']]
city_ages_ratings

,age,rating
city,,
Boston,51.86,6.57
Buffalo,47.89,5.00
Chicago,45.50,5.25
Orlando,41.29,6.71
Phoenix,44.67,5.33
Pittsburgh,49.38,4.88


But what if we didn't drop the customer_id duplicates...would we get the same city result??

In [35]:
city_ages_ratings = demo_ratings.groupby('city').mean().round(decimals=2)[['age', 'rating']]
city_ages_ratings

,age,rating
city,,
Boston,51.00,6.20
Buffalo,53.08,4.69
Chicago,38.83,4.00
Orlando,43.38,6.50
Phoenix,44.58,5.50
Pittsburgh,49.38,4.88


What would be the total mean for the US customers?

In [39]:
us_avg = demo_ratings_drop_dups.describe().round(2)['rating']['mean']
us_avg

5.59

##### Breaking Down the Inner Join <br>
<p>Let's take a closer look at the actual mechanics here however. There is some subtlety and nuance that is best understood by thinking of an inner join as two steps: a cartesian product, and then a filter. Observe the result of the following inner join:</p>

In [40]:
d1 = {
    'A': [1, 2, 3, 4, 2],
    'B': ['a', 'b', 'c', 'd', 'x']
}

df1 = pd.DataFrame.from_dict(d1)

df1

,A,B
0,1,a
1,2,b
2,3,c
3,4,d
4,2,x


In [41]:
d2 = {
    'A': [1, 2, 6, 2, 2],
    'C': ['e', 'f', 'g', 'h', 'y']
}

df2 = pd.DataFrame.from_dict(d2)

df2

,A,C
0,1,e
1,2,f
2,6,g
3,2,h
4,2,y


In [47]:
df1.merge(df2, how='inner', on='A')

,A,B,C
0,1,a,e
1,2,b,f
2,2,b,h
3,2,b,y
4,2,x,f
5,2,x,h
6,2,x,y


<p>Somehow, we ended up with more rows in the result than either of the original tables! This is because we joined on a $\textit{non-unique index}$. The join starts by computing all the possible combinations of rows, and then filters them based on the condition. Because of the duplicates, multiple combinations of rows make it through!</p>

### Outer Joins <br>
<p>An $\textit{outer join}$ produces all of the rows from the left table, and joins whatever rows it can to the right table, filling anywhere it cant with NaN's or None values. Below is a brief example, and then we will discuss a possible use case for this type of join.</p>

In [ ]:
df3

##### Use Case Example

<p>As an example use case, let's imagine we have again the demographics and survey response tables from above:</p>

<p>Now, you are asked as an analyst to compare the demographics of survey responders to the demographics of all your customers combined.</p>

In [ ]:
# Count all survey responders against the total number of customers we have 
# (Specifically the customers that haven't taken the survey yet)



# Producing similar result with .count()


In [ ]:
# average age of people that answered the survey


# demo_outer_surveys['customer_rating'] != 'N/A'


In [ ]:
# find the average age of people that did not answer the survey


In [ ]:
# Pip install for both psycopg2 and SQLAlchemy
import sys
!{sys.executable} -m pip install psycopg2 SQLAlchemy

In [ ]:
# databaselanguage(postgresql)://user:password@url:port/database_name


In [ ]:
# Writing a DataFrame to PostGreSQL database 
# from sqlalchemy.types import Integer, Text,String,DateTime,Float
